# 5.3 공식문서의 MCP Client 활용방법 I (feat. MultiServerMCPClient)

- 3.8 커스텀 도구(tool)를 최대한 활용하는 방법에서 생성한 에이전트가 활용하는 도구를 [MCP 서버](https://github.com/modelcontextprotocol/servers)로 이전



In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
query = "5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?"

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
)

In [4]:
async def process_stream(stream_generator):
    results = []
    try:
        async for chunk in stream_generator:

            key = list(chunk.keys())[0]

            if key == "agent":
                # Agent 메시지의 내용을 가져옴. 메세지가 비어있는 경우 어떤 도구를 어떻게 호출할지 정보를 가져옴
                content = (
                    chunk["agent"]["messages"][0].content
                    if chunk["agent"]["messages"][0].content != ""
                    else chunk["agent"]["messages"][0].additional_kwargs
                )
                print(f"'agent': '{content}'")

            elif key == "tools":
                # 도구 메시지의 내용을 가져옴
                for tool_msg in chunk["tools"]["messages"]:
                    print(f"'tools': '{tool_msg.content}'")

            results.append(chunk)
        return results
    except Exception as e:
        print(f"Error processing stream: {e}")
        return results

- MCP 서버와 연결하기 위한 MCP 클라이언트 생성
    - LangChain에서 제공하는 [langchain-mcp-adapters](https://github.com/langchain-ai/langchain-mcp-adapters)활용
    - `MultiServerMCPClient`를 활용하면 복수의 MCP 서버와 연결할 수 있음

In [5]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

async with MultiServerMCPClient(
    {
        "house_tax": {
            "command": "python",
            "args": ["./mcp_stdio_server.py"],
            "transport": "stdio",
        },
    }
) as client:

    ##### AGENT #####
    tool_list = client.get_tools()
    agent = create_react_agent(llm, tool_list)
    query = "5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?"
    system_prompt = client.get_prompt(
        server_name="house_tax", prompt_name="house_tax_system_prompt", arguments={}
    )
    messages = await system_prompt + [HumanMessage(content=query)]
    print("=====RESPONSE=====")
    stream_generator = agent.astream({"messages": messages})

    all_chunks = await process_stream(stream_generator)

    if all_chunks:
        final_result = all_chunks[-1]
        print("\nFinal result:", final_result)

NotImplementedError: As of langchain-mcp-adapters 0.1.0, MultiServerMCPClient cannot be used as a context manager (e.g., async with MultiServerMCPClient(...)). Instead, you can do one of the following:
1. client = MultiServerMCPClient(...)
   tools = await client.get_tools()
2. client = MultiServerMCPClient(...)
   async with client.session(server_name) as session:
       tools = await load_mcp_tools(session)

- 현재 안됨
- 확인 필요